# **Импотр библиотек**

In [1]:
# перед началом работы импортируем библиотеки, которые могут понадобиться в работе над проектом

import pandas as pd
import math
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats as st
import seaborn as sns
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import requests 
from urllib.parse import urlencode 
import re
from calendar import day_name

%matplotlib inline
# импорт библиотеки подключения к яндекс-метрике
from tapi_yandex_metrika import YandexMetrikaStats
from pandas.io.json import json_normalize
import json
from requests_html import HTMLSession
session = HTMLSession()
from pandas import Series,DataFrame
from io import BytesIO
from time import sleep
import sys

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

## Функция запроса к директу (req_to_direct), ввод тела и заголовков отдельно

In [3]:
def req_to_direct(URL, body, headers):   
    # --- Запуск цикла для выполнения запросов ---
    # Если получен HTTP-код 200, то выводится содержание отчета
    # Если получен HTTP-код 201 или 202, выполняются повторные запросы
    while True:
        try:
            req = requests.post(URL, body, headers=headers)
            req.encoding = 'utf-8'  # Принудительная обработка ответа в кодировке UTF-8
            if req.status_code == 400:
                print("Параметры запроса указаны неверно или достигнут лимит отчетов в очереди")
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код запроса: {}".format(u(body)))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break
            elif req.status_code == 200:
                print("Отчет создан успешно")
                format(u(req.text))
                break
            elif req.status_code == 201:
                print("Отчет успешно поставлен в очередь в режиме офлайн")
                retryIn = int(req.headers.get("retryIn", 60))
                print("Повторная отправка запроса через {} секунд".format(retryIn))
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                sleep(retryIn)
            elif req.status_code == 202:
                print("Отчет формируется в режиме офлайн")
                retryIn = int(req.headers.get("retryIn", 60))
                print("Повторная отправка запроса через {} секунд".format(retryIn))
                print("RequestId:  {}".format(req.headers.get("RequestId", False)))
                sleep(retryIn)
            elif req.status_code == 500:
                print("При формировании отчета произошла ошибка. Пожалуйста, попробуйте повторить запрос позднее")
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break
            elif req.status_code == 502:
                print("Время формирования отчета превысило серверное ограничение.")
                print("Пожалуйста, попробуйте изменить параметры запроса - уменьшить период и количество запрашиваемых данных.")
                print("JSON-код запроса: {}".format(body))
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break
            else:
                print("Произошла непредвиденная ошибка")
                print("RequestId:  {}".format(req.headers.get("RequestId", False)))
                print("JSON-код запроса: {}".format(body))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break

        # Обработка ошибки, если не удалось соединиться с сервером API Директа
        except ConnectionError:
            # В данном случае мы рекомендуем повторить запрос позднее
            print("Произошла ошибка соединения с сервером API")
            # Принудительный выход из цикла
            break

        # Если возникла какая-либо другая ошибка
        except:
            # В данном случае мы рекомендуем проанализировать действия приложения
            print("Произошла непредвиденная ошибка")
            # Принудительный выход из цикла
            break
    return(req.text)
    # return(req.content) если вдруг хотим в байтах выгрузить и преобразовывать
                

## Функция запроса к директу (rep), ввод токена 

In [51]:
# ФУЦНКЦИЯ ПОЛУЧЕНИЯ ДАННЫХ ИЗ ДИРЕКТА
# НА ВХОД ПОДАЮТСЯ ТОКЕН И ЛОГИН КЛИЕНТА, ДАТЫ НАЧАЛА И ОКОНЧАНИЯ ОТЧЕТА
# ЕСЛИ НУЖНА КОНВЕРСИЯ ТО ЦЕЛИ УКАЗЫВАЮТСЯ ВНУТРИ ФУНКЦИИ
def rep(token,login,date_from,date_to):
    if sys.version_info < (3,):
        def u(x):
            try:
                return x.encode("utf8")
            except UnicodeDecodeError:
                return x
    else:
        def u(x):
            if type(x) == type(b''):
                return x.decode('utf8')
            else:
                return x

    # --- Входные данные ---
    # Адрес сервиса Reports для отправки JSON-запросов (регистрозависимый)
    ReportsURL = 'https://api-sandbox.direct.yandex.com/json/v5/reports'

    # OAuth-токен пользователя, от имени которого будут выполняться запросы
    token = token

    # Логин клиента рекламного агентства
    # Обязательный параметр, если запросы выполняются от имени рекламного агентства
    #clientLogin = login
    
    #ID цели
    goalID = ['43693399'] # если целей несколько то ['123','324']
    
    stolbec = ["Date", "AdNetworkType", "CampaignName", "AdGroupName", "Impressions", "Clicks", "Cost", "AvgEffectiveBid", "AvgPageviews",
           "Bounces", "Sessions",  "Conversions", "Placement", "Device", "ExternalNetworkName", "LocationOfPresenceName"]
    
    stolbec_alt = ["Date", "CampaignName", "LocationOfPresenceName", "Impressions", "Clicks", "Cost"]

    # --- Подготовка запроса ---
    # Создание HTTP-заголовков запроса
    headers = {
        # OAuth-токен. Использование слова Bearer обязательно
        "Authorization": "Bearer " + token,
        # Логин клиента рекламного агентства
        #"Client-Login": clientLogin,
        # Язык ответных сообщений
        "Accept-Language": "ru",
        # Режим формирования отчета
        "processingMode": "auto",
        # Формат денежных значений в отчете
         "returnMoneyInMicros": "false",
        # Не выводить в отчете строку с названием отчета и диапазоном дат
        # "skipReportHeader": "true",
        # Не выводить в отчете строку с названиями полей
        # "skipColumnHeader": "true",
        # Не выводить в отчете строку с количеством строк статистики
         "skipReportSummary": "true"
    }

        
    # Создание тела запроса
    body = {
        "params": {
            "SelectionCriteria": {
                "DateFrom": date_from,
                "DateTo": date_to
            },
            
            # цель, если требуется
            #"Goals": goalID, 
            
            "FieldNames": stolbec,
            
            "ReportName": u("Report9"),
            "ReportType": "CRITERIA_PERFORMANCE_REPORT",
            "DateRangeType": "CUSTOM_DATE",
            "Format": "TSV",
            "IncludeVAT": "YES",
            "IncludeDiscount": "NO"
        }
    }

    # Кодирование тела запроса в JSON
    body = json.dumps(body, indent=4)

    # --- Запуск цикла для выполнения запросов ---
    # Если получен HTTP-код 200, то выводится содержание отчета
    # Если получен HTTP-код 201 или 202, выполняются повторные запросы
    while True:
        try:
            req = requests.post(ReportsURL, body, headers=headers)
            req.encoding = 'utf-8'  # Принудительная обработка ответа в кодировке UTF-8
            if req.status_code == 400:
                print("Параметры запроса указаны неверно или достигнут лимит отчетов в очереди")
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код запроса: {}".format(u(body)))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break
            elif req.status_code == 200:

                format(u(req.text))
                break
            elif req.status_code == 201:
                print("Отчет успешно поставлен в очередь в режиме офлайн")
                retryIn = int(req.headers.get("retryIn", 60))
                print("Повторная отправка запроса через {} секунд".format(retryIn))
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                sleep(retryIn)
            elif req.status_code == 202:
                print("Отчет формируется в режиме офлайн")
                retryIn = int(req.headers.get("retryIn", 60))
                print("Повторная отправка запроса через {} секунд".format(retryIn))
                print("RequestId:  {}".format(req.headers.get("RequestId", False)))
                sleep(retryIn)
            elif req.status_code == 500:
                print("При формировании отчета произошла ошибка. Пожалуйста, попробуйте повторить запрос позднее")
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break
            elif req.status_code == 502:
                print("Время формирования отчета превысило серверное ограничение.")
                print(
                    "Пожалуйста, попробуйте изменить параметры запроса - уменьшить период и количество запрашиваемых данных.")
                print("JSON-код запроса: {}".format(body))
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break
            else:
                print("Произошла непредвиденная ошибка")
                print("RequestId:  {}".format(req.headers.get("RequestId", False)))
                print("JSON-код запроса: {}".format(body))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break

        # Обработка ошибки, если не удалось соединиться с сервером API Директа
        except ConnectionError:
            # В данном случае мы рекомендуем повторить запрос позднее
            print("Произошла ошибка соединения с сервером API")
            # Принудительный выход из цикла
            break

        # Если возникла какая-либо другая ошибка
        except:
            # В данном случае мы рекомендуем проанализировать действия приложения
            print("Произошла непредвиденная ошибка")
            # Принудительный выход из цикла
            break

    json_string = json.dumps(body)
    return req.text

## Ссылка на спецификации запросов директа

https://yandex.ru/dev/direct/doc/reports/spec.html

https://yandex.ru/dev/direct/doc/reports/fields-list.html

In [5]:
# токен доступа к директу
ACCESS_TOKEN = "y0_AgAAAAADW_NUAAj-6gAAAADZkOljtoPASx4mRcSALt07Rr16GnaguH8"


In [6]:
#токены
mytoken=ACCESS_TOKEN

#логины
project = 'elama-99999999'

#Даты
DateFrom="2022-01-01"
DateTo="2022-02-02"

In [52]:
data=rep(mytoken,project,DateFrom,DateTo)

file=open("cashe.csv","w")
file.write(data)
file.close()
f=pd.read_csv("cashe.csv", header=1, sep='\t', index_col=0, encoding='cp1251') 
f.head()

Отчет успешно поставлен в очередь в режиме офлайн
Повторная отправка запроса через 1 секунд
RequestId: 4916163861770184878
Отчет формируется в режиме офлайн
Повторная отправка запроса через 10 секунд
RequestId:  4916165541847128845
Отчет формируется в режиме офлайн
Повторная отправка запроса через 10 секунд
RequestId:  4916177051616524651
Отчет успешно поставлен в очередь в режиме офлайн
Повторная отправка запроса через 1 секунд
RequestId: 4916189120471741139
Параметры запроса указаны неверно или достигнут лимит отчетов в очереди
RequestId: 4916190701765439769
JSON-код запроса: {
    "params": {
        "SelectionCriteria": {
            "DateFrom": "2022-01-01",
            "DateTo": "2022-02-02"
        },
        "FieldNames": [
            "Date",
            "AdNetworkType",
            "CampaignName",
            "AdGroupName",
            "Impressions",
            "Clicks",
            "Cost",
            "AvgEffectiveBid",
            "AvgPageviews",
            "Bounces",
   

ParserError: Passed header=1 but only 1 lines in file

In [56]:
# --- Входные данные --- ЕСЛИ НЕ ИСПОЛЬЗУЕТСЯ  ФУНКЦИЯ "rep"
# Адрес сервиса Reports для отправки JSON-запросов (регистрозависимый)
ReportsURL = 'https://api.direct.yandex.com/json/v5/reports'

#адрес сервиса ОТЧЕТОВ ПЕСОЧНИЦА
ReportsURL_SANDBOX = 'https://api-sandbox.direct.yandex.com/json/v5/reports'

# OAuth-токен пользователя, от имени которого будут выполняться запросы
token = ACCESS_TOKEN

# Логин клиента рекламного агентства
# Обязательный параметр, если запросы выполняются от имени рекламного агентства
#clientLogin = 'ЛОГИН_КЛИЕНТА'

#какие столбцы подгрузить
stolbec = ["Date", "AdNetworkType", "CampaignName", "AdGroupName", "AdId", "Impressions", "Clicks", "Cost", "AvgEffectiveBid", "AvgPageviews",
           "Bounces", "Sessions",  "Conversions", "Placement"]

#ID цели
goalID = ['43693399'] # если целей несколько то ['123','324']

#даты отчета
firstDate = "2022-05-01"
secondDate = "2022-08-15"

# --- Подготовка запроса ---
# Создание HTTP-заголовков запроса
headers = {
           # OAuth-токен. Использование слова Bearer обязательно
           "Authorization": "Bearer " + token,
           # Логин клиента рекламного агентства
          # "Client-Login": clientLogin,
           # Язык ответных сообщений
           "Accept-Language": "ru",
           # Режим формирования отчета
           "processingMode": "auto",
           # Формат денежных значений в отчете, если не сделать, то расходы будут в миллионах
            "returnMoneyInMicros": "false",
           # Не выводить в отчете строку с названием отчета и диапазоном дат
           # "skipReportHeader": "true",
           # Не выводить в отчете строку с названиями полей
           # "skipColumnHeader": "true",
           # Не выводить в отчете строку с количеством строк статистики
            "skipReportSummary": "true"
           }

# Создание тела запроса
body = {
    "params": {
        # критерии выбора, тут могут быть фильтры, логические операторы, 
        "SelectionCriteria": {
            "DateFrom": "2022-01-01",
            "DateTo": "2022-12-01"
                             },
        # цель, если требуется
        #"Goals": goalID, 
        
        "FieldNames": stolbec,
        "ReportName": ("ОПyuuШЩ6767"),
        "ReportType": "CUSTOM_REPORT",   # тип отчета
        "DateRangeType": "CUSTOM_DATE",
        "Format": "TSV",  # обязательно
        "IncludeVAT": "YES",
        "IncludeDiscount": "NO"
              }
       }

# Кодирование тела запроса в JSON
body = json.dumps(body, indent=4)


In [57]:
result = req_to_direct(ReportsURL_SANDBOX, body, headers)

# Преобразование ответа в датафрейм   ()      
file = open("cashe.csv", "w")   # сздаем и открываем новый файл
file.write(result)            # записываем в него ответ API
file.close()                    # закрываем файл
df = pd.read_csv("cashe.csv", header=1, sep='\t', index_col=0, encoding='cp1251')     # записываем в датафрейм файл
df = df.reset_index()
df.head()

Отчет успешно поставлен в очередь в режиме офлайн
Повторная отправка запроса через 1 секунд
RequestId: 4916472231831477239
Отчет формируется в режиме офлайн
Повторная отправка запроса через 10 секунд
RequestId:  4916473868960516248
Отчет формируется в режиме офлайн
Повторная отправка запроса через 10 секунд
RequestId:  4916485594220392409
Отчет формируется в режиме офлайн
Повторная отправка запроса через 10 секунд
RequestId:  4916497091847883546
Отчет формируется в режиме офлайн
Повторная отправка запроса через 10 секунд
RequestId:  4916508615245260960
Отчет формируется в режиме офлайн
Повторная отправка запроса через 10 секунд
RequestId:  4916520108577488761
Отчет формируется в режиме офлайн
Повторная отправка запроса через 10 секунд
RequestId:  4916531614051693090
Отчет формируется в режиме офлайн
Повторная отправка запроса через 10 секунд
RequestId:  4916543012892687817
Отчет успешно поставлен в очередь в режиме офлайн
Повторная отправка запроса через 1 секунд
RequestId: 49165547861

ParserError: Passed header=1 but only 1 lines in file

In [23]:
# --- Входные данные ---
# Адрес сервиса Reports для отправки JSON-запросов (регистрозависимый)
ReportsURL = 'https://api.direct.yandex.com/json/v5/reports'

# адрес сервиса ОТЧЕТОВ ПЕСОЧНИЦА
ReportsURL_SANDBOX = 'https://api-sandbox.direct.yandex.com/json/v5/reports'

# OAuth-токен пользователя, от имени которого будут выполняться запросы
token = ACCESS_TOKEN

# Логин клиента рекламного агентства
# Обязательный параметр, если запросы выполняются от имени рекламного агентства
#clientLogin = 'ЛОГИН_КЛИЕНТА'

# какие столбцы подгрузить
stolbec = ["Date", "AdNetworkType", "CampaignName", "AdGroupName", "AdId", "Impressions", "Clicks", "Cost", "AvgEffectiveBid", "AvgPageviews",
           "Bounces", "Sessions",  "Conversions", "Placement", "Device", "ExternalNetworkName", "LocationOfPresenceName"]

# ID цели
goalID = ['43693399']  # если целей несколько то ['123','324']


# --- Подготовка запроса ---
# Создание HTTP-заголовков запроса
headers = {
    # OAuth-токен. Использование слова Bearer обязательно
    "Authorization": "Bearer " + token,
    # Логин клиента рекламного агентства
    # "Client-Login": clientLogin,
    # Язык ответных сообщений
    "Accept-Language": "ru",
    # Режим формирования отчета
    "processingMode": "auto",
    # Формат денежных значений в отчете, если не сделать, то расходы будут в миллионах
    "returnMoneyInMicros": "false",
    # Не выводить в отчете строку с названием отчета и диапазоном дат
    # "skipReportHeader": "true",
    # Не выводить в отчете строку с названиями полей
    # "skipColumnHeader": "true",
    # Не выводить в отчете строку с количеством строк статистики
    "skipReportSummary": "true"
}

# Создание тела запроса
body = {
    "params": {
        # критерии выбора, тут могут быть фильтры, логические операторы,
        "SelectionCriteria": {
            "DateFrom": "2022-01-01",
            "DateTo": "2022-02-01"
        },
        # цель, если требуется
        # "Goals": goalID,

        "FieldNames": stolbec,
        "ReportName": ("Отчет 92"),
        "ReportType": "ADGRPUOP_PERFORMANCE_REPORT",   # тип отчета
        "DateRangeType": "CUSTOM_DATE",
        "Format": "TSV",  # обязательно
        "IncludeVAT": "YES",
        "IncludeDiscount": "NO"
    }
}

# Кодирование тела запроса в JSON
body = json.dumps(body, indent=4)

In [24]:
result = req_to_direct(ReportsURL_SANDBOX, body, headers)

# Преобразование ответа в датафрейм   ()      
file = open("cashe.csv", "w")   # сздаем и открываем новый файл
file.write(result)            # записываем в него ответ API
file.close()                    # закрываем файл
df = pd.read_csv("cashe.csv", header=1, sep='\t', index_col=0, encoding='cp1251')     # записываем в датафрейм файл
df = df.reset_index()
df.head()

Отчет успешно поставлен в очередь в режиме офлайн
Повторная отправка запроса через 1 секунд
RequestId: 4914311313528169983
Отчет формируется в режиме офлайн
Повторная отправка запроса через 10 секунд
RequestId:  4914312890524392119
Отчет формируется в режиме офлайн
Повторная отправка запроса через 10 секунд
RequestId:  4914324413178235240
Отчет успешно поставлен в очередь в режиме офлайн
Повторная отправка запроса через 1 секунд
RequestId: 4914336280916537553
Отчет формируется в режиме офлайн
Повторная отправка запроса через 10 секунд
RequestId:  4914337977428705671
Отчет формируется в режиме офлайн
Повторная отправка запроса через 10 секунд
RequestId:  4914349435654265299
Отчет успешно поставлен в очередь в режиме офлайн
Повторная отправка запроса через 1 секунд
RequestId: 4914361294060255457
Отчет формируется в режиме офлайн
Повторная отправка запроса через 10 секунд
RequestId:  4914362887492829179
Отчет формируется в режиме офлайн
Повторная отправка запроса через 10 секунд
RequestId

ParserError: Passed header=1 but only 1 lines in file

In [12]:
# ОБРАЗЕЦ ОБРАЩЕНИЯ ОТ ЯНДЕКСА
# -*- coding: utf-8 -*-
import requests
from requests.exceptions import ConnectionError
from time import sleep
import json

# Метод для корректной обработки строк в кодировке UTF-8 как в Python 3, так и в Python 2
import sys

if sys.version_info < (3,):
    def u(x):
        try:
            return x.encode("utf8")
        except UnicodeDecodeError:
            return x
else:
    def u(x):
        if type(x) == type(b''):
            return x.decode('utf8')
        else:
            return x

# --- Входные данные ---
# Адрес сервиса Reports для отправки JSON-запросов (регистрозависимый)
ReportsURL = 'https://api-sandbox.direct.yandex.com/json/v5/reports'

# OAuth-токен пользователя, от имени которого будут выполняться запросы
token = ACCESS_TOKEN

# Логин клиента рекламного агентства
# Обязательный параметр, если запросы выполняются от имени рекламного агентства
clientLogin = 'ЛОГИН_КЛИЕНТА'

# --- Подготовка запроса ---
# Создание HTTP-заголовков запроса
headers = {
           # OAuth-токен. Использование слова Bearer обязательно
           "Authorization": "Bearer " + token,
           # Логин клиента рекламного агентства
           #"Client-Login": clientLogin,
           # Язык ответных сообщений
           "Accept-Language": "ru",
           # Режим формирования отчета
           "processingMode": "auto"
           # Формат денежных значений в отчете
           # "returnMoneyInMicros": "false",
           # Не выводить в отчете строку с названием отчета и диапазоном дат
           # "skipReportHeader": "true",
           # Не выводить в отчете строку с названиями полей
           # "skipColumnHeader": "true",
           # Не выводить в отчете строку с количеством строк статистики
           # "skipReportSummary": "true"
           }

# Создание тела запроса
body = {
    "params": {
        "SelectionCriteria": {
            "DateFrom": "2022-01-01",
            "DateTo": "2022-02-03"
        },
        "FieldNames": [
            "Date",
            "CampaignName",
            "LocationOfPresenceName",
            "Impressions",
            "Clicks",
            "Cost"
        ],
        "ReportName": u(";ljhlkjhglkhj"),
        "ReportType": "CAMPAIGN_PERFORMANCE_REPORT",
        "DateRangeType": "CUSTOM_DATE",
        "Format": "TSV",
        "IncludeVAT": "NO",
        "IncludeDiscount": "NO"
    }
}

# Кодирование тела запроса в JSON
body = json.dumps(body, indent=4)

# --- Запуск цикла для выполнения запросов ---
# Если получен HTTP-код 200, то выводится содержание отчета
# Если получен HTTP-код 201 или 202, выполняются повторные запросы
while True:
    try:
        req = requests.post(ReportsURL, body, headers=headers)
        req.encoding = 'utf-8'  # Принудительная обработка ответа в кодировке UTF-8
        if req.status_code == 400:
            print("Параметры запроса указаны неверно или достигнут лимит отчетов в очереди")
            print("RequestId: {}".format(req.headers.get("RequestId", False)))
            print("JSON-код запроса: {}".format(u(body)))
            print("JSON-код ответа сервера: \n{}".format(u(req.json())))
            break
        elif req.status_code == 200:
            print("Отчет создан успешно")
            print("RequestId: {}".format(req.headers.get("RequestId", False)))
            print("Содержание отчета: \n{}".format(u(req.text)))
            break
        elif req.status_code == 201:
            print("Отчет успешно поставлен в очередь в режиме офлайн")
            retryIn = int(req.headers.get("retryIn", 60))
            print("Повторная отправка запроса через {} секунд".format(retryIn))
            print("RequestId: {}".format(req.headers.get("RequestId", False)))
            sleep(retryIn)
        elif req.status_code == 202:
            print("Отчет формируется в режиме офлайн")
            retryIn = int(req.headers.get("retryIn", 60))
            print("Повторная отправка запроса через {} секунд".format(retryIn))
            print("RequestId:  {}".format(req.headers.get("RequestId", False)))
            sleep(retryIn)
        elif req.status_code == 500:
            print("При формировании отчета произошла ошибка. Пожалуйста, попробуйте повторить запрос позднее")
            print("RequestId: {}".format(req.headers.get("RequestId", False)))
            print("JSON-код ответа сервера: \n{}".format(u(req.json())))
            break
        elif req.status_code == 502:
            print("Время формирования отчета превысило серверное ограничение.")
            print("Пожалуйста, попробуйте изменить параметры запроса - уменьшить период и количество запрашиваемых данных.")
            print("JSON-код запроса: {}".format(body))
            print("RequestId: {}".format(req.headers.get("RequestId", False)))
            print("JSON-код ответа сервера: \n{}".format(u(req.json())))
            break
        else:
            print("Произошла непредвиденная ошибка")
            print("RequestId:  {}".format(req.headers.get("RequestId", False)))
            print("JSON-код запроса: {}".format(body))
            print("JSON-код ответа сервера: \n{}".format(u(req.json())))
            break

    # Обработка ошибки, если не удалось соединиться с сервером API Директа
    except ConnectionError:
        # В данном случае мы рекомендуем повторить запрос позднее
        print("Произошла ошибка соединения с сервером API")
        # Принудительный выход из цикла
        break

    # Если возникла какая-либо другая ошибка
    except:
        # В данном случае мы рекомендуем проанализировать действия приложения
        print("Произошла непредвиденная ошибка")
        # Принудительный выход из цикла
        break

Отчет создан успешно
RequestId: 4913416453540772717
Содержание отчета: 
";ljhlkjhglkhj (2022-01-01 - 2022-02-03)"
Date	CampaignName	LocationOfPresenceName	Impressions	Clicks	Cost
2022-01-01	Test API Sandbox campaign 1	Москва и область	74020	6745	4299790000
2022-01-02	Test API Sandbox campaign 1	Москва и область	71485	6785	3772880000
2022-01-03	Test API Sandbox campaign 1	Москва и область	69010	7610	4728810000
2022-01-04	Test API Sandbox campaign 1	Москва и область	76855	6925	4407200000
2022-01-05	Test API Sandbox campaign 1	Москва и область	84455	5115	3321610000
2022-01-06	Test API Sandbox campaign 1	Москва и область	74675	6575	4234320000
2022-01-07	Test API Sandbox campaign 1	Москва и область	75795	6725	4813980000
2022-01-08	Test API Sandbox campaign 1	Москва и область	67750	6495	3936230000
2022-01-09	Test API Sandbox campaign 1	Москва и область	75695	7040	4206990000
2022-01-10	Test API Sandbox campaign 1	Москва и область	68830	6785	4577540000
2022-01-11	Test API Sandbox campaign 1	Мо

In [ ]:
# ВАРИАНТ ДЛЯ ВЫГРУЗКИ В БАЙТАХ, НЕОБХОДИМО МЕНЯТЬ СКРИПТ И ВОЗВРАТ ФУНКЦИИ
# присваеваем результат в переменную
#result = req_to_direct(ReportsURL_SANDBOX, body, headers)
# записываем его в байтах
#file_from_bytes = BytesIO(result.content)
# преобразуем в датафрейм
#df = pd.read_csv(file_from_bytes, sep='\t', header=1)
# корректируем значение столбца стоимости при необходимости
# df['Cost'] = df['Cost'] / 1000000 
#df.head()

## *Функция группировки данных по одному параметру (group_df)*

In [ ]:
def group_df(df, column):
    # сгруппируем количество показов, кликов, СТР и стоимости по устройствам
    grouped_device = df.groupby(column).agg({'visits': ['sum'], 'bounceRate': ['mean'], 
                                             'pageDepth': ['mean'], 'avgVisitSec': ['mean'], 'Goals_total': ['sum']}).reset_index()
    grouped_device.columns = [column, 'Визитов', '% откзов', 'Ср глубина просмотра', 'Ср сессия в сек', 'Достигнуто целей']
    
    grouped_device['Конверсия'] = (grouped_device['Достигнуто целей'] / grouped_device['Визитов']) * 100
    for col in ('% откзов', 'Ср глубина просмотра', 'Ср сессия в сек', 'Конверсия'):
        grouped_device[col] = round((grouped_device[col]), 2)
    
    return(grouped_device)

## *Функция группировки по двум пар-м (group_df_2_param)*

In [ ]:
def group_df_2_param(df, column1, column2):
    # сгруппируем количество показов, кликов, СТР и стоимости по устройствам
    grouped_device = df.groupby([column1, column2] ).agg({'visits': ['sum'], 'bounceRate': ['mean'], 
                                             'pageDepth': ['mean'], 'avgVisitSec': ['mean'], 'Goals_total': ['sum']}).reset_index()
    grouped_device.columns = [column1, column2, 'Визитов', '% откзов', 'Ср глубина просмотра', 'Ср сессия в сек', 'Достигнуто целей']

    grouped_device['Конверсия'] = (grouped_device['Достигнуто целей'] / grouped_device['Визитов']) * 100
    for col in ('% откзов', 'Ср глубина просмотра', 'Ср сессия в сек', 'Конверсия'):
        grouped_device[col] = round((grouped_device[col]), 2)
        
    return(grouped_device)